# Dataset overview

In this notebook we review class counts in train and validation splits

In [1]:
import wandb
import pathlib
import shutil
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from typing import List


def load_raw_image_data(run) -> pathlib.Path:
    """
    Unpacks data from an artifact into a folder and returns the path to the folder.
    """

    artifact_name = f"letters_splits"
    artifact = run.use_artifact(f"master-thesis/{artifact_name}:latest")
    artifact_dir = pathlib.Path(
        f"./artifacts/{artifact.name.replace(':', '-')}"
    ).resolve()
    if not artifact_dir.exists():
        artifact_dir = artifact.download()
        artifact_dir = pathlib.Path(artifact_dir).resolve()
        for split_file in artifact_dir.iterdir():
            if split_file.name.endswith(".tar.gz"):
                split = split_file.name.replace(".tar.gz", "")
                shutil.unpack_archive(split_file, artifact_dir / split, format="gztar")

    return [artifact_dir / split for split in ["train", "test", "val"]]

def load_tfdataset_data(run) -> List[tf.data.Dataset]:
    """
    Downloads datasets from a wandb artifact and loads them into a list of tf.data.Datasets.
    """

    artifact_name = f"letters_splits_tfds"
    artifact = run.use_artifact(f"master-thesis/{artifact_name}:latest")
    artifact_dir = pathlib.Path(
        f"./artifacts/{artifact.name.replace(':', '-')}"
    ).resolve()
    if not artifact_dir.exists():
        artifact_dir = artifact.download()
        artifact_dir = pathlib.Path(artifact_dir).resolve()

    # if tf.__version__ minor is less than 10, use
    # tf.data.experimental.load instead of tf.data.Dataset.load

    if int(tf.__version__.split(".")[1]) < 10:
        load_function = tf.data.experimental.load
    else:
        load_function = tf.data.Dataset.load
    
    output_list = []
    for split in ["train", "test", "val"]:
        ds = load_function(str(artifact_dir / split), compression="GZIP")
        output_list.append(ds)
    
    return output_list

2023-01-22 16:42:31.186380: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
char_id_to_class_name = {
        0: "0",
        1: "1",
        2: "2",
        3: "3",
        4: "4",
        5: "5",
        6: "6",
        7: "7",
        8: "8",
        9: "9",
        10: "a",
        11: "b",
        12: "c",
        13: "d",
        14: "e",
        15: "f",
        16: "g",
        17: "h",
        18: "i",
        19: "j",
        20: "k",
        21: "l",
        22: "m",
        23: "n",
        24: "o",
        25: "p",
        26: "q",
        27: "r",
        28: "s",
        29: "t",
        30: "u",
        31: "v",
        32: "w",
        33: "x",
        34: "y",
        35: "z",
        36: "A",
        37: "B",
        38: "C",
        39: "D",
        40: "E",
        41: "F",
        42: "G",
        43: "H",
        44: "I",
        45: "J",
        46: "K",
        47: "L",
        48: "M",
        49: "N",
        50: "O",
        51: "P",
        52: "Q",
        53: "R",
        54: "S",
        55: "T",
        56: "U",
        57: "V",
        58: "W",
        59: "X",
        60: "Y",
        61: "Z",
        # then lowercase letters of the Polish alphabet: ą, ć, ę, ł, ń, ó, ś, ź, ż
        62: "ą",
        63: "ć",
        64: "ę",
        65: "ł",
        66: "ń",
        67: "ó",
        68: "ś",
        69: "ź",
        70: "ż",
        # then uppercase letters of the Polish alphabet: Ą, Ć, Ę, Ł, Ń, Ó, Ś, Ź, Ż
        71: "Ą",
        72: "Ć",
        73: "Ę",
        74: "Ł",
        75: "Ń",
        76: "Ó",
        77: "Ś",
        78: "Ź",
        79: "Ż",
        # then special characters: + - : ; $ ! ? @
        80: "+",
        81: "-",
        82: ":",
        83: ";",
        84: "$",
        85: "!",
        86: "?",
        87: "@",
        88: ".",
    }

In [4]:
run = wandb.init(project="master-thesis", job_type="preprocessing")

wandb: Currently logged in as: gratkadlafana. Use `wandb login --relogin` to force relogin


In [ ]:
# load data from raw images (.png files)
split_paths = load_raw_image_data(run=run)

ds_train = tf.keras.utils.image_dataset_from_directory(
        split_paths[0],
        image_size=(32, 32),
        color_mode="grayscale",
    )

ds_test = tf.keras.utils.image_dataset_from_directory(
        split_paths[1],
        image_size=(32, 32),
        color_mode="grayscale",
    )

ds_val = tf.keras.utils.image_dataset_from_directory(
        split_paths[2],
        image_size=(32, 32),
        color_mode="grayscale",
    )

number_of_classes = len(ds_train.class_names)

# save datasets on disk then upload to wandb as artifacts

output_dir = pathlib.Path("./datasets").resolve()
output_dir.mkdir(exist_ok=True)

ds_train.save(str(output_dir / "train"), compression="GZIP")
ds_val.save(str(output_dir / "val"), compression="GZIP")
ds_test.save(str(output_dir / "test"), compression="GZIP")

artifact = wandb.Artifact("letters_splits_tfds", type="dataset", description="Dataset splits in tf.data.Dataset format")
artifact.add_dir(output_dir)
run.log_artifact(artifact)

In [5]:
# load data from tf.data.Dataset artifacts

ds_train, ds_test, ds_val = load_tfdataset_data(run)

wandb: Downloading large artifact letters_splits_tfds:latest, 54.88MB. 18 files... 
wandb:   18 of 18 files downloaded.  
Done. 0:0:0.0
2023-01-22 16:42:55.959841: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-22 16:42:55.963268: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-22 16:42:55.963520: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-22 16:42:55.963816: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network L

## Dataset profiling

In [8]:
! pip install tensorflow-datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 12.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.5/146.5 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.9/152.9 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.0/223.0 kB 15.4 MB/s eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21485 sha256=44464148027313ca52cecd7ddaee3c7539caf3d3372b8d4442a9df01f0811902
  Stored in directory: /root/.cache/pip/wheels/6a/fe/dc/a7b3e03dfd0afb3a19691905bbafac1fbaebb704a02a4daeb2
Successfully built promise


In [9]:
import tensorflow_datasets as tfds

def preprocess_dataset(ds: tf.data.Dataset, batch_size: int, cache: bool = True) -> tf.data.Dataset:
    ds = ds.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))  # normalize
    ds = ds.unbatch().batch(batch_size)
    if cache:
        ds = ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

ds_train_not_cached = preprocess_dataset(ds_train, 128, cache=False)

tfds.benchmark(ds_train_not_cached, batch_size=128)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



************ Summary ************



1377it [00:01, 1300.47it/s]

Examples/sec (First included) 162587.17 ex/sec (total: 176384 ex, 1.08 sec)
Examples/sec (First only) 4062.55 ex/sec (total: 128 ex, 0.03 sec)
Examples/sec (First excluded) 167328.88 ex/sec (total: 176256 ex, 1.05 sec)


,duration,num_examples,avg
first+lasts,1.084858,176384,162587.166667
first,0.031507,128,4062.549036
lasts,1.053351,176256,167328.877389


In [16]:
ds_train_cached = preprocess_dataset(ds_train, 128*4, cache=True)

tfds.benchmark(ds_train_cached, batch_size=128*2)


************ Summary ************



345it [00:01, 316.74it/s]

Examples/sec (First included) 79659.94 ex/sec (total: 88576 ex, 1.11 sec)
Examples/sec (First only) 8668.86 ex/sec (total: 256 ex, 0.03 sec)
Examples/sec (First excluded) 81596.79 ex/sec (total: 88320 ex, 1.08 sec)


,duration,num_examples,avg
first+lasts,1.111927,88576,79659.936250
first,0.029531,256,8668.856162
lasts,1.082396,88320,81596.786058


In [17]:
tfds.benchmark(ds_train_cached, batch_size=128*4)


************ Summary ************



345it [00:00, 5092.07it/s]

Examples/sec (First included) 2502097.77 ex/sec (total: 177152 ex, 0.07 sec)
Examples/sec (First only) 114783.97 ex/sec (total: 512 ex, 0.00 sec)
Examples/sec (First excluded) 2662613.38 ex/sec (total: 176640 ex, 0.07 sec)


,duration,num_examples,avg
first+lasts,0.070801,177152,2.502098e+06
first,0.004461,512,1.147840e+05
lasts,0.066341,176640,2.662613e+06


In [19]:
# save datasets on disk then upload to wandb as artifacts

output_dir = pathlib.Path("./datasets").resolve()
output_dir.mkdir(exist_ok=True)

ds_train.save(str(output_dir / "train"), compression="GZIP")
ds_val.save(str(output_dir / "val"), compression="GZIP")
ds_test.save(str(output_dir / "test"), compression="GZIP")

artifact = wandb.Artifact("letters_splits_tfds", type="dataset", description="Dataset splits in tf.data.Dataset format")
artifact.add_dir(output_dir)
run.log_artifact(artifact)

wandb: Adding directory to artifact (/home/wflis/code/master-thesis/notebooks/datasets)... Done. 0.1s


In [ ]:
import math

CLASSES = list(char_id_to_class_name.keys())

# TFRecord util functions

def decode_jpeg_and_label(filename):
  bits = tf.io.read_file(filename)
  image = tf.io.decode_jpeg(bits)
  # parse flower name from containing directory
  label = tf.strings.split(tf.expand_dims(filename, axis=-1), sep='/')
  label = label.values[-2]
  return image, label

def recompress_image(image, label):
  height = tf.shape(image)[0]
  width = tf.shape(image)[1]
  image = tf.cast(image, tf.uint8)
  image = tf.image.encode_jpeg(image, optimize_size=True, chroma_downsampling=False)
  return image, label, height, width

# Three types of data can be stored in TFRecords: bytestrings, integers and floats
# They are always stored as lists, a single data element will be a list of size 1

def _bytestring_feature(list_of_bytestrings):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=list_of_bytestrings))

def _int_feature(list_of_ints): # int64
  return tf.train.Feature(int64_list=tf.train.Int64List(value=list_of_ints))

def _float_feature(list_of_floats): # float32
  return tf.train.Feature(float_list=tf.train.FloatList(value=list_of_floats))

def to_tfrecord(tfrec_filewriter, img_bytes, label, height, width):  
  class_num = np.argmax(np.array(CLASSES)==label) # 'roses' => 2 (order defined in CLASSES)
  one_hot_class = np.eye(len(CLASSES))[class_num]     # [0, 0, 1, 0, 0] for class #2, roses

  feature = {
      "image": _bytestring_feature([img_bytes]), # one image in the list
      "class": _int_feature([class_num]),        # one class in the list
      
      # additional (not very useful) fields to demonstrate TFRecord writing/reading of different types of data
      "label":         _bytestring_feature([label]),          # fixed length (1) list of strings, the text label
      "size":          _int_feature([height, width]),         # fixed length (2) list of ints
      "one_hot_class": _float_feature(one_hot_class.tolist()) # variable length  list of floats, n=len(CLASSES)
  }
  return tf.train.Example(features=tf.train.Features(feature=feature))

def write_tfrecords(split_path, data_dir, shards=16, output_dir="./datasets_tfrecords"):
  output_dir = pathlib.Path(output_dir).resolve()
  output_dir.mkdir(exist_ok=True)

  DATA_OUTPUT = str(output_dir) 
  DATA_PATTERN = f"{split_path}/*/*.png" 
  SHARDS = shards
  AUTOTUNE = tf.data.AUTOTUNE

  nb_images = len(tf.io.gfile.glob(DATA_PATTERN))
  shard_size = math.ceil(1.0 * nb_images / SHARDS)
  print("Pattern matches {} images which will be rewritten as {} .tfrec files containing {} images each.".format(nb_images, SHARDS, shard_size))

  filenames = tf.data.Dataset.list_files(DATA_PATTERN, seed=35155) # This also shuffles the images
  dataset1 = filenames.map(decode_jpeg_and_label, num_parallel_calls=AUTOTUNE)

  dataset3 = dataset1.map(recompress_image, num_parallel_calls=AUTOTUNE)
  dataset3 = dataset3.batch(shard_size) # sharding: there will be one "batch" of images per file 

  print("Writing TFRecords")
  for shard, (image, label, height, width) in enumerate(dataset3):
    # batch size used as shard size here
    shard_size = image.numpy().shape[0]
    # good practice to have the number of records in the filename
    filename = DATA_OUTPUT + "{:02d}-{}.tfrec".format(shard, shard_size)
    
    with tf.io.TFRecordWriter(filename) as out_file:
      for i in range(shard_size):
        example = to_tfrecord(out_file,
                              image.numpy()[i], # re-compressed image: already a byte string
                              label.numpy()[i],
                              height.numpy()[i],
                              width.numpy()[i])
        out_file.write(example.SerializeToString())
      print("Wrote file {} containing {} records".format(filename, shard_size))

In [ ]:
output_dir = pathlib.Path("./datasets_tfrecords").resolve()
output_dir.mkdir(exist_ok=True)

shards=16
output_dir="./datasets_tfrecords"

split_path = split_paths[0]
DATA_OUTPUT = str(output_dir) 
DATA_PATTERN = f"{split_path}/*/*.png" 
SHARDS = shards
AUTOTUNE = tf.data.AUTOTUNE

nb_images = len(tf.io.gfile.glob(DATA_PATTERN))
shard_size = math.ceil(1.0 * nb_images / SHARDS)
print("Pattern matches {} images which will be rewritten as {} .tfrec files containing {} images each.".format(nb_images, SHARDS, shard_size))

In [ ]:
filenames = tf.data.Dataset.list_files(DATA_PATTERN, seed=35155) # This also shuffles the images
dataset1 = filenames.map(decode_jpeg_and_label, num_parallel_calls=AUTOTUNE)

In [ ]:
dataset3 = dataset1.map(recompress_image, num_parallel_calls=AUTOTUNE)
dataset3 = dataset3.batch(shard_size) # sharding: there will be one "batch" of images per file 

In [ ]:
print("Writing TFRecords")
for shard, (image, label, height, width) in enumerate(dataset3):
    # batch size used as shard size here
    shard_size = image.numpy().shape[0]
    # good practice to have the number of records in the filename
    filename = DATA_OUTPUT + "/{:02d}-{}.tfrec".format(shard, shard_size)


    with tf.io.TFRecordWriter(filename) as out_file:
        for i in range(shard_size):
            '''
            example = to_tfrecord(out_file,
                                    image.numpy()[i], # re-compressed image: already a byte string
                                    label.numpy()[i],
                                    height.numpy()[i],
                                    width.numpy()[i])
            '''
            #out_file.write(example.SerializeToString())
            print("Wrote file {} containing {} records".format(filename, shard_size))

In [ ]:
# save as TFRecords and upload to WandB

tfrecord_dir = "./datasets_tfrecords"
for split_path in split_paths:
  write_tfrecords(split_path, tfrecord_dir)

artifact = wandb.Artifact("letters_splits_tfds", type="dataset", description="Dataset splits in tf.data.TFRecord format")
artifact.add_dir(tfrecord_dir)
run.log_artifact(artifact)

In [ ]:
# calculate class count for each split
train_class_count = np.zeros(number_of_classes)
for _, label in ds_train:
    train_class_count += tf.math.bincount(label, minlength=number_of_classes)

val_class_count = np.zeros(number_of_classes)
for _, label in ds_val:
    val_class_count += tf.math.bincount(label, minlength=number_of_classes)

# plot class count for each split
plt.bar(ds_train.class_names, train_class_count)
plt.title("Train")
plt.show()

plt.bar(ds_val.class_names, val_class_count)

# log class count for each split to wandb

wandb.log({"train_class_count": wandb.Histogram(train_class_count)})
wandb.log({"val_class_count": wandb.Histogram(val_class_count)})
